In [77]:
%load_ext autoreload
%autoreload 2
import os
import sys
from pathlib import Path
import networkx as nx
import pandas as pd
import numpy as np
import scipy.stats as stats
sys.path.insert(0, Path.cwd().absolute())

BASE_DIR = (Path.cwd()).parent

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
from network_analysis.flow_assignment import FlowAssignment
from network_analysis.geodf_network import GeoDataNetwork
from geoprocessing.faf_zone_network import FafZoneNetwork
from geoprocessing.temperature_overlay.overlay_spear import join_tas_diff
from network_analysis.mc_flow_assignment import sample_flows
import geopandas as gpd

In [79]:
SIMPLE_NET = False # (421, 423, 341, 342 )

MONTE_CARLO = False
MC_ITER_N = 500
NORMAL_DIST = True

COLUMN = 'thousand tons in 2035'

# Validations
if SIMPLE_NET:
  MONTE_CARLO = False
  

In [80]:
naRailNodesDf = gpd.read_file( BASE_DIR / "inputs/NTAD_North_American_Rail_Network_Nodes")
naRailLinesDf = gpd.read_file( BASE_DIR / "inputs/NTAD_North_American_Rail_Network_Lines")
fafZonesDf = gpd.read_file( BASE_DIR / "inputs/faf_zones_exploded").to_crs(4326)


In [81]:
fzn = FafZoneNetwork(naRailNodesDf, naRailLinesDf, fafZonesDf).setInputDfColType()
faf_flows = pd.read_csv(Path.cwd().parent / 'inputs/faf_freight_flow_exploded.csv' )
railnet = fzn.createNetwork()

fa = FlowAssignment(railnet)

railnet_flows = fa(faf_flows, SUM_COLUMN=COLUMN) # For comparison, if Monte Carlo mode

In [83]:
fa = FlowAssignment(railnet)

railnet_flows = fa(faf_flows, SUM_COLUMN=COLUMN) # For comparison, if Monte Carlo mode

if not MONTE_CARLO:
  fzn.apply_flows(railnet_flows.to_dict())
  # fzn.apply_flows(link_weights(fzn.fafZoneLinksDf))
if MONTE_CARLO:
  monte_railnet_flows = []
  monte_inputs = []
  for i in range(MC_ITER_N):
    iteration_inputs = sample_flows(faf_flows, COLUMN, distribution='normal' if NORMAL_DIST else 'triangular')
    monte_inputs.append(iteration_inputs)
    faf_flows['input_flows'] = iteration_inputs
    iteration_flow = fa.run(faf_flows, SUM_COLUMN='input_flows')
    monte_railnet_flows.append(iteration_flow)
  monte_railnet_flows = pd.concat(monte_railnet_flows, axis=1)
  monte_railnet_flows.columns=range(MC_ITER_N)
  fzn.fafZoneLinksDf['flows'] = monte_railnet_flows.mean(axis=1)
  fzn.fafZoneLinksDf['flow_covs'] = monte_railnet_flows.std(axis=1) / monte_railnet_flows.mean(axis=1)
  railnet_flows.index = railnet_flows.index.set_names({'seg_start':'FAF_Zone_fr', 'seg_end':'FAF_Zone_to'})
  fzn.fafZoneLinksDf['flow_df'] = (fzn.fafZoneLinksDf['flows'] - railnet_flows).abs()
  fzn.fafZoneLinksDf['flow_df_percent'] = fzn.fafZoneLinksDf['flow_df'] / railnet_flows

# Unweighted Network

In [84]:
faf_geonet = GeoDataNetwork(
  fzn.fafZoneNodesDf.copy(),
  fzn.fafZoneLinksDf.copy(),
  'FAF_Zone_1',
  # lambda df : df['flows'],
  lambda df : 1,
  lambda df : df.reset_index().set_index(['FAF_Zone_fr', 'FAF_Zone_to'], drop=False)[['FAF_Zone_fr', 'FAF_Zone_to']],
  VALUE='centrality'
)

faf_geonet.calculate_centrality(weighted=False)
faf_railnet = faf_geonet.createNetwork(NAME='FAF')
faf_geonet.create_normalizers(stats.uniform, stats.uniform)
display(faf_geonet.show_map())
faf_geonet.show_values()

,FAF_Zone_1,centrality
FAF_Zone,,
219,Remainder of Kentucky,0.465028
179,Remainder of Illinois,0.358933
429,Remainder of Pennsylvania,0.245991
299,Remainder of Missouri,0.226317
489,Remainder of Texas,0.161161
540,West Virginia,0.148282
369,Remainder of New York,0.141527
519,Remainder of Virginia,0.130659
2399,Remainder of Ohio,0.126260


FAF_Zone_1_fr  \
FAF_Zone_fr FAF_Zone_to                                     
179         219                     Remainder of Illinois   
            299                     Remainder of Illinois   
369         429                     Remainder of New York   
219         540                     Remainder of Kentucky   
            2399                    Remainder of Kentucky   
361         369          Albany-Schenectady, NY  CFS Area   
429         540                 Remainder of Pennsylvania   
179         190                     Remainder of Illinois   
49          350                      Remainder of Arizona   
429         2399                Remainder of Pennsylvania   

                                     FAF_Zone_1_to  centrality  
FAF_Zone_fr FAF_Zone_to                                         
179         219              Remainder of Kentucky    0.278785  
            299              Remainder of Missouri    0.198187  
369         429          Remainder of Pennsylvania    0.148452  
219         540                      West Virginia    0.135070  
            2399                 Remainder of Ohio    0.118641  
361         369              Remainder of New York    0.113553  
429         540                      West Virginia    0.109560  
179         190                               Iowa    0.109054  
49          350                         New Mexico    0.108332  
429         2399                 Remainder of Ohio    0.105765

# Freight Tonnage Weighted

## Freight Tonnage

In [92]:
faf_geonet = GeoDataNetwork(
  fzn.fafZoneNodesDf.copy(),
  fzn.fafZoneLinksDf.copy(),
  'FAF_Zone_1',
  lambda df : 0,
  lambda df : df.reset_index().set_index(['FAF_Zone_fr', 'FAF_Zone_to'], drop=False)[['FAF_Zone_fr', 'FAF_Zone_to']],
  VALUE='flows'
)

faf_geonet.calculate_centrality()
faf_railnet = faf_geonet.createNetwork(NAME='FAF')
faf_geonet.create_normalizers(stats.gengamma, stats.uniform)
display(faf_geonet.show_map())
faf_geonet.show_values()

,FAF_Zone_1,flows
FAF_Zone,,
61,"Los Angeles-Long Beach, CA CFS Area",2.348032e+07
49,Remainder of Arizona,1.282123e+07
219,Remainder of Kentucky,7.499376e+06
179,Remainder of Illinois,7.442690e+06
4069,Remainder of California,6.984260e+06
363,"New York-Newark, NY-NJ-CT-PA CFS Area (NY Part)",6.533530e+06
190,Iowa,6.045752e+06
369,Remainder of New York,4.518809e+06
319,Remainder of Nebraska,3.943476e+06


FAF_Zone_1_fr  \
FAF_Zone_fr FAF_Zone_to                                                     
49          61                                       Remainder of Arizona   
61          4069                     Los Angeles-Long Beach, CA  CFS Area   
363         369          New York-Newark, NY-NJ-CT-PA  CFS Area (NY Part)   
61          63                       Los Angeles-Long Beach, CA  CFS Area   
92          363          New York-Newark, NY-NJ-CT-PA  CFS Area (CT Part)   
190         319                                                      Iowa   
179         190                                     Remainder of Illinois   
            219                                     Remainder of Illinois   
261         269                    Detroit-Warren-Ann Arbor, MI  CFS Area   
61          321                      Los Angeles-Long Beach, CA  CFS Area   

                                                            FAF_Zone_1_to  \
FAF_Zone_fr FAF_Zone_to                                                     
49          61                       Los Angeles-Long Beach, CA  CFS Area   
61          4069                                  Remainder of California   
363         369                                     Remainder of New York   
61          63                           San Diego-Carlsbad, CA  CFS Area   
92          363          New York-Newark, NY-NJ-CT-PA  CFS Area (NY Part)   
190         319                                     Remainder of Nebraska   
179         190                                                      Iowa   
            219                                     Remainder of Kentucky   
261         269                                     Remainder of Michigan   
61          321            Las Vegas-Henderson, NV-AZ  CFS Area (NV Part)   

                                flows  
FAF_Zone_fr FAF_Zone_to                
49          61           2.391455e+07  
61          4069         1.315985e+07  
363         369          6.544189e+06  
61          63           5.495002e+06  
92          363          5.371918e+06  
190         319          4.818200e+06  
179         190          4.729298e+06  
            219          4.624769e+06  
261         269          4.231779e+06  
61          321          3.752480e+06

## Node / Link importances

In [86]:
faf_geonet = GeoDataNetwork(
  fzn.fafZoneNodesDf.copy(),
  fzn.fafZoneLinksDf.copy(),
  'FAF_Zone_1',
  lambda df : df['flows'],
  # lambda df : 0,
  lambda df : df.reset_index().set_index(['FAF_Zone_fr', 'FAF_Zone_to'], drop=False)[['FAF_Zone_fr', 'FAF_Zone_to']],
  VALUE='centrality'
)

faf_geonet.calculate_centrality()
faf_railnet = faf_geonet.createNetwork(NAME='FAF')
faf_geonet.create_normalizers(stats.uniform, stats.uniform)
display(faf_geonet.show_map())
faf_geonet.show_values()

,FAF_Zone_1,centrality
FAF_Zone,,
1479,Remainder of Tennessee,0.342444
540,West Virginia,0.325373
209,Remainder of Kansas,0.315826
219,Remainder of Kentucky,0.315693
3479,Remainder of Tennessee,0.293954
489,Remainder of Texas,0.293439
472,"Nashville-Davidson--Murfreesboro, TN CFS Area",0.287792
471,"Memphis-Forrest City, TN-MS-AR CFS Area (TN P...",0.266185
223,"New Orleans-Metairie-Hammond, LA-MS CFS Area ...",0.256440


,,FAF_Zone_1_fr,FAF_Zone_1_to,centrality
FAF_Zone_fr,FAF_Zone_to,,,
472,3479,"Nashville-Davidson--Murfreesboro, TN CFS Area",Remainder of Tennessee,0.281527
219,472,Remainder of Kentucky,"Nashville-Davidson--Murfreesboro, TN CFS Area",0.279937
471,3479,"Memphis-Forrest City, TN-MS-AR CFS Area (TN P...",Remainder of Tennessee,0.270840
219,1479,Remainder of Kentucky,Remainder of Tennessee,0.266669
209,319,Remainder of Kansas,Remainder of Nebraska,0.261313
280,471,Mississippi,"Memphis-Forrest City, TN-MS-AR CFS Area (TN P...",0.259188
223,280,"New Orleans-Metairie-Hammond, LA-MS CFS Area ...",Mississippi,0.255604
249,540,Remainder of Maryland,West Virginia,0.249935
223,229,"New Orleans-Metairie-Hammond, LA-MS CFS Area ...",Remainder of Louisiana,0.249896


# Temperature

In [87]:
# fzn.fafZoneNodesDf['tas'] = join_tas_diff(fzn.fafZoneNodesDf.drop(columns=['index_right']))
fzn._fafZoneLinksDf = None
fzn.fafZoneLinksDf['tas'] = join_tas_diff(fzn.fafZoneLinksDf)

fa = FlowAssignment(fzn.createNetwork())

railnet_flows = fa(faf_flows, SUM_COLUMN=COLUMN) # For comparison, if Monte Carlo mode

fzn.apply_flows(railnet_flows.to_dict())


geometry  \
FAF_Zone_fr FAF_Zone_to                                                      
11          19           LINESTRING (-86.76365 33.47538, -86.78008 33.0...   
12          19           LINESTRING (-88.06251 30.72613, -86.78008 33.0...   
            280          LINESTRING (-88.06251 30.72613, -89.48377 32.5...   
            4129         LINESTRING (-88.06251 30.72613, -84.7611 30.22...   
19          131          LINESTRING (-86.78008 33.0568, -84.37138 33.80...   
...                                                                    ...   
531         539          LINESTRING (-122.37721 47.46371, -119.26865 47...   
532         539          LINESTRING (-122.76488 45.88636, -119.26865 47...   
540         2399         LINESTRING (-81.02183 38.58522, -81.5012 40.06...   
551         559          LINESTRING (-88.19888 43.0759, -89.63998 44.41...   
1479        3379         LINESTRING (-82.64762 36.4054, -82.06291 35.64...   

                         FRAARCID                  geometry_fr COUNTRY_fr  \
FAF_Zone_fr FAF_Zone_to                                                     
11          19                 19   POINT (-86.76365 33.47538)         US   
12          19                  2   POINT (-88.06251 30.72613)         US   
            280                 2   POINT (-88.06251 30.72613)         US   
            4129                2   POINT (-88.06251 30.72613)         US   
19          131                 4    POINT (-86.78008 33.0568)         US   
...                           ...                          ...        ...   
531         539                 6  POINT (-122.37721 47.46371)         US   
532         539                 1  POINT (-122.76488 45.88636)         US   
540         2399                7   POINT (-81.02183 38.58522)         US   
551         559                15    POINT (-88.19888 43.0759)         US   
1479        3379                1    POINT (-82.64762 36.4054)         US   

                        STATE_fr STFIPS_fr CTYFIPS_fr STCYFIPS_fr  \
FAF_Zone_fr FAF_Zone_to                                             
11          19                AL        01        127       01127   
12          19                AL        01        097       01097   
            280               AL        01        097       01097   
            4129              AL        01        097       01097   
19          131               AL        01        023       01023   
...                          ...       ...        ...         ...   
531         539               WA        53        045       53045   
532         539               WA        53        015       53015   
540         2399              WV        54        099       54099   
551         559               WI        55        055       55055   
1479        3379              TN        47        013       47013   

                         FRADISTRCT_fr PASSNGR_fr  ...  \
FAF_Zone_fr FAF_Zone_to                            ...   
11          19                       3          A  ...   
12          19                       3          A  ...   
            280                      3          A  ...   
            4129                     3          A  ...   
19          131                      3          A  ...   
...                                ...        ...  ...   
531         539                      8          A  ...   
532         539                      8          A  ...   
540         2399                     2          A  ...   
551         559                      4          A  ...   
1479        3379                     3       None  ...   

                                                             FAF_Zone_1_to  \
FAF_Zone_fr FAF_Zone_to                                                      
11          19                                        Remainder of Alabama   
12          19                                        Remainder of Alabama   
            280                                                Mississippi   
      

geometry  \
FAF_Zone_fr FAF_Zone_to                                                      
11          19           LINESTRING (-86.76365 33.47538, -86.78008 33.0...   
12          19           LINESTRING (-88.06251 30.72613, -86.78008 33.0...   
            280          LINESTRING (-88.06251 30.72613, -89.48377 32.5...   
            4129         LINESTRING (-88.06251 30.72613, -84.7611 30.22...   
19          131          LINESTRING (-86.78008 33.0568, -84.37138 33.80...   
...                                                                    ...   
531         539          LINESTRING (-122.37721 47.46371, -119.26865 47...   
532         539          LINESTRING (-122.76488 45.88636, -119.26865 47...   
540         2399         LINESTRING (-81.02183 38.58522, -81.5012 40.06...   
551         559          LINESTRING (-88.19888 43.0759, -89.63998 44.41...   
1479        3379         LINESTRING (-82.64762 36.4054, -82.06291 35.64...   

                         FRAARCID                  geometry_fr COUNTRY_fr  \
FAF_Zone_fr FAF_Zone_to                                                     
11          19                 19   POINT (-86.76365 33.47538)         US   
12          19                  2   POINT (-88.06251 30.72613)         US   
            280                 2   POINT (-88.06251 30.72613)         US   
            4129                2   POINT (-88.06251 30.72613)         US   
19          131                 4    POINT (-86.78008 33.0568)         US   
...                           ...                          ...        ...   
531         539                 6  POINT (-122.37721 47.46371)         US   
532         539                 1  POINT (-122.76488 45.88636)         US   
540         2399                7   POINT (-81.02183 38.58522)         US   
551         559                15    POINT (-88.19888 43.0759)         US   
1479        3379                1    POINT (-82.64762 36.4054)         US   

                        STATE_fr STFIPS_fr CTYFIPS_fr STCYFIPS_fr  \
FAF_Zone_fr FAF_Zone_to                                             
11          19                AL        01        127       01127   
12          19                AL        01        097       01097   
            280               AL        01        097       01097   
            4129              AL        01        097       01097   
19          131               AL        01        023       01023   
...                          ...       ...        ...         ...   
531         539               WA        53        045       53045   
532         539               WA        53        015       53015   
540         2399              WV        54        099       54099   
551         559               WI        55        055       55055   
1479        3379              TN        47        013       47013   

                         FRADISTRCT_fr PASSNGR_fr  ... FAF_Zone_D_to  \
FAF_Zone_fr FAF_Zone_to                            ...                 
11          19                       3          A  ...    Rest of AL   
12          19                       3          A  ...    Rest of AL   
            280                      3          A  ...   Mississippi   
            4129                     3          A  ...    Rest of FL   
19          131                      3          A  ...    Atlanta GA   
...                                ...        ...  ...           ...   
531         539                      8          A  ...    Rest of WA   
532         539                      8          A  ...    Rest of WA   
540         2399                     2          A  ...    Rest of OH   
551         559                      4          A  ...    Rest of WI   
1479        3379                     3       None  ...    Rest of NC   

                                                             FAF_Zone_1_to  \
FAF_Zone_fr FAF_Zone_to                                                      
11          19                                        Rema

In [88]:
faf_geonet = GeoDataNetwork(
  fzn.fafZoneNodesDf.copy(),
  fzn.fafZoneLinksDf.copy(),
  'FAF_Zone_1',
  # lambda df : df['flows'],
  lambda df : 1,
  lambda df : df.reset_index().set_index(['FAF_Zone_fr', 'FAF_Zone_to'], drop=False)[['FAF_Zone_fr', 'FAF_Zone_to']],
  VALUE='tas'
)

faf_railnet = faf_geonet.createNetwork(NAME='FAF')
faf_geonet.create_normalizers(stats.uniform, stats.uniform)
display(faf_geonet.show_map(color_nodes=True))
faf_geonet.show_values()

KeyError: ['tas']

In [ ]:
faf_geonet = GeoDataNetwork(
  fzn.fafZoneNodesDf.copy(),
  fzn.fafZoneLinksDf.copy(),
  'FAF_Zone_1',
  lambda df : df['tas'],
  lambda df : df.reset_index().set_index(['FAF_Zone_fr', 'FAF_Zone_to'], drop=False)[['FAF_Zone_fr', 'FAF_Zone_to']],
  VALUE='centrality'
)

faf_geonet.calculate_centrality(weighted=True)
faf_railnet = faf_geonet.createNetwork(NAME='FAF')
faf_geonet.create_normalizers(stats.uniform, stats.uniform)
display(faf_geonet.show_map())
faf_geonet.show_values()

,FAF_Zone_1,centrality
FAF_Zone,,
11,"Birmingham-Hoover-Talladega, AL CFS Area",0.000000
12,"Mobile-Daphne-Fairhope, AL CFS Area",0.036919
19,Remainder of Alabama,0.106421
41,"Phoenix-Mesa-Scottsdale, AZ CFS Area",0.000000
42,"Tucson-Nogales, AZ CFS Area",0.000000
...,...,...
3399,Remainder of Ohio,0.017455
3479,Remainder of Tennessee,0.183116
4069,Remainder of California,0.026447


FAF_Zone_1_fr  \
FAF_Zone_fr FAF_Zone_to                                                      
11          19                   Birmingham-Hoover-Talladega, AL  CFS Area   
12          19                        Mobile-Daphne-Fairhope, AL  CFS Area   
            280                       Mobile-Daphne-Fairhope, AL  CFS Area   
            4129                      Mobile-Daphne-Fairhope, AL  CFS Area   
19          131                                       Remainder of Alabama   
...                                                                    ...   
531         539                               Seattle-Tacoma, WA  CFS Area   
532         539          Portland-Vancouver-Salem, OR-WA  CFS Area (WA ...   
540         2399                                             West Virginia   
551         559                    Milwaukee-Racine-Waukesha, WI  CFS Area   
1479        3379                                    Remainder of Tennessee   

                                                             FAF_Zone_1_to  \
FAF_Zone_fr FAF_Zone_to                                                      
11          19                                        Remainder of Alabama   
12          19                                        Remainder of Alabama   
            280                                                Mississippi   
            4129                                      Remainder of Florida   
19          131          Atlanta-Athens-Clarke County-Sandy Springs, GA...   
...                                                                    ...   
531         539                                    Remainder of Washington   
532         539                                    Remainder of Washington   
540         2399                                         Remainder of Ohio   
551         559                                   Remainder of Wisconsin\r   
1479        3379                               Remainder of North Carolina   

                         centrality  
FAF_Zone_fr FAF_Zone_to              
11          19             0.014388  
12          19             0.006569  
            280            0.042331  
            4129           0.038265  
19          131            0.012303  
...                             ...  
531         539            0.014180  
532         539            0.013033  
540         2399           0.000938  
551         559            0.000834  
1479        3379           0.014493  

[275 rows x 3 columns]

In [ ]:
from utils import normalize_reverse

faf_geonet = GeoDataNetwork(
  fzn.fafZoneNodesDf.copy(),
  fzn.fafZoneLinksDf.copy(),
  'FAF_Zone_1',
  lambda df : normalize_reverse(df['tas']) * normalize_reverse(df['flows']),
  lambda df : df.reset_index().set_index(['FAF_Zone_fr', 'FAF_Zone_to'], drop=False)[['FAF_Zone_fr', 'FAF_Zone_to']],
  VALUE='centrality'
)

faf_geonet.calculate_centrality(weighted=True)
faf_railnet = faf_geonet.createNetwork(NAME='FAF')
faf_geonet.create_normalizers(stats.uniform, stats.uniform)
display(faf_geonet.show_map())
faf_geonet.show_values()

,FAF_Zone_1,centrality
FAF_Zone,,
219,Remainder of Kentucky,0.500159
179,Remainder of Illinois,0.416958
540,West Virginia,0.272929
429,Remainder of Pennsylvania,0.246483
319,Remainder of Nebraska,0.183011
559,Remainder of Wisconsin\r,0.178673
369,Remainder of New York,0.162382
279,Remainder of Minnesota,0.151169
380,North Dakota,0.150376


FAF_Zone_1_fr  \
FAF_Zone_fr FAF_Zone_to                                     
179         219                     Remainder of Illinois   
219         540                     Remainder of Kentucky   
49          61                       Remainder of Arizona   
429         540                 Remainder of Pennsylvania   
369         429                     Remainder of New York   
279         380                    Remainder of Minnesota   
            559                    Remainder of Minnesota   
179         190                     Remainder of Illinois   
361         369          Albany-Schenectady, NY  CFS Area   
179         559                     Remainder of Illinois   

                                                FAF_Zone_1_to  centrality  
FAF_Zone_fr FAF_Zone_to                                                    
179         219                         Remainder of Kentucky    0.372745  
219         540                                 West Virginia    0.237775  
49          61           Los Angeles-Long Beach, CA  CFS Area    0.229330  
429         540                                 West Virginia    0.218069  
369         429                     Remainder of Pennsylvania    0.168543  
279         380                                  North Dakota    0.152539  
            559                      Remainder of Wisconsin\r    0.149984  
179         190                                          Iowa    0.139871  
361         369                         Remainder of New York    0.135492  
179         559                      Remainder of Wisconsin\r    0.133615

In [ ]:
import matplotlib.pyplot as plt

import folium
import branca.colormap as cm
from IPython.display import display


min_lat=24.7433195
max_lat=49.3457868
min_lon=-124.7844079
max_lon=-66.9513812


m = folium.Map(
  max_bounds=True,
  location=[42, -95],
  zoom_start=4,
  min_zoom=4,
  tiles='OpenStreetMap',
  # max_lat=max_lat,
  # min_lon=min_lon,
  # max_lon=max_lon,
  # min_lat=min_lat
)


# flows.apply(distrib.cdf).plot(kind='hist', bins=50)

m.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])

lineWeightMax = 5
lineWeightMin = 0.5

if True:
  zoneDf = fzn.fafZonesDf.copy()
  zoneDf.geometry = zoneDf.geometry.simplify(.15)
  zoneDf[['FAF_Zone_1', 'geometry']].explore(
    m=m,
    column='FAF_Zone_1',
    cmap='Greens',
    legend=False,
    style_kwds={'opacity':.1}
  )

if False: 
  from folium import DivIcon
  for i, r in fzn.fafZoneLinksDf.iterrows():
    center = r.geometry.centroid
    folium.map.Marker(
      [center.y, center.x],
      icon=DivIcon(
          icon_size=(20,150),
          icon_anchor=(0,0),
          html='<div style="font-size: 12px">%s</div>' % round(r.flows,4),
        )
    ).add_to(m)


  for i, r in fzn.fafZoneNodesDf.iterrows():
    center = r.geometry.centroid
    folium.map.Marker(
      [center.y, center.x],
      icon=DivIcon(
          icon_size=(20,150),
          icon_anchor=(0,0),
          html='<div style="font-size: 16px; font-weight: bold">%s</div>' % int(i),
        )
    ).add_to(m)

get_flow = lambda feature : feature['properties']['flows']
fzn.fafZoneLinksDf[['FAF_Zone_1_fr', 'FAF_Zone_1_to', 'flows', 'geometry']].explore(
  m=m,
  column='flows',
  style_kwds={
    'style_function': lambda feature: {
      'color': cm.linear.plasma(link_normalizer(get_flow(feature))),
      'weight': link_normalizer(get_flow(feature)) * (lineWeightMax - lineWeightMin) + lineWeightMin
    }
  },
  legend=True,
  legend_kwds={
    'legend_name': 'Tons of Freight Transported',
  }
)


fzn.fafZoneNodesDf[['FAF_Zone_1', 'flows', 'geometry']].explore(
  m=m,
  color='black',
  # column='flows',
  style_kwds={
    'style_function': lambda feature: {
      # 'color': cm.linear.plasma(norm_w(feature)),
      'weight': node_normalizer(get_flow(feature)) * (lineWeightMax - lineWeightMin) + lineWeightMin
    }
  }
)
display(m)
if MONTE_CARLO:
  fzn.fafZoneLinksDf['flow_df%'] = (fzn.fafZoneLinksDf['flow_df_percent'] * 100).round(2)
fzn.fafZoneLinksDf['flowr'] = fzn.fafZoneLinksDf['flows'].round()

display(fzn.fafZoneLinksDf.sort_values('flows', axis=0, ascending=False).reset_index()[
  ['FAF_Zone_1_fr', 'FAF_Zone_1_to', 'flowr'] + (['flow_df%', 'flow_covs', 'flow_df', ] if MONTE_CARLO else [])
].head(10))
fzn.fafZoneLinksDf['flows'].plot(
  kind='hist',
  title='Histogram of freight flows',
  xlabel='Millions of Tons per year'
)
plt.show()

if MONTE_CARLO:
  from matplotlib.ticker import PercentFormatter

  diff_percents = monte_railnet_flows.sub(railnet_flows, axis=0).abs().div(railnet_flows, axis=0).mul(100, axis=0)
  plot = diff_percents.T.expanding().mean().abs().mean(axis=1).plot(
    title='Monte Carlo Simulation Convergence',
    xlabel='Iterations',
    ylabel='Precent Error'
  )
  plot.yaxis.set_major_formatter(PercentFormatter())
  plt.show()
  avg_diff_percents = diff_percents.mean(axis=1)
  fzn.fafZoneLinksDf.loc[~np.isfinite(fzn.fafZoneLinksDf.flow_df_percent), 'flow_df_percent'] = 0
  plot = avg_diff_percents.plot(
    kind='hist',
    title='Histogram of flow errors in percent',
    xlabel='Percent Error'
  )
  mean_error = avg_diff_percents.mean()
  plt.axvline(mean_error, color='k')
  plot.xaxis.set_major_formatter(PercentFormatter())
  
  min_ylim, max_ylim = plt.ylim()
  plt.text(mean_error * 1.1, max_ylim*0.9, 'Mean: {:.4f}%'.format(mean_error))
  plt.show()

NameError: name 'link_normalizer' is not defined